In [19]:
import numpy as np

In [30]:
class DenseLayer():
    def __init__(self, number_of_neurons, number_of_inputs_per_neuron):
        self.synaptic_weights = 2 * np.random.random((number_of_inputs_per_neuron, number_of_neurons)) - 1

In [36]:
class NN():
    def __init__(self, layers, learnrate):
        # seed so random numbers are the same
        np.random.seed(1)
        
        self.weights = layers
            
        # create layer outputs
        self.layer_outputs = []
        
        # set learn rate
        self.learnrate = learnrate
            
        print([x.synaptic_weights.shape for x in self.weights])
        

    def activate(self, x, deriv=False):
        if deriv:
            return 1 - (np.tanh(x))**2
        else:
            return np.tanh(x)
    
    def forward_prop(self, data):
        # compute layer outputs
        self.layer_outputs = [data]
        for i in range(len(self.weights)):
            self.layer_outputs.append(self.activate(np.dot(self.layer_outputs[i], self.weights[i].synaptic_weights)))
        return self.layer_outputs[-1]
    
    def train(self, training_set_inputs, training_set_outputs, epochs):
        for e in range(epochs):
            # Pass the training set through our neural network
            output = self.forward_prop(training_set_inputs)

            # Calculate the error for layer 2 (The difference between the desired output
            # and the predicted output).
            output_error = training_set_outputs - output
            output_delta = output_error * self.activate(output, deriv=True)

            # Calculate the error for layer 1 (By looking at the weights in layer 1,
            # we can determine by how much layer 1 contributed to the error in layer 2).
            prev_delta = output_delta
            layer_adjustments = []
            for layer in range(len(self.weights)-2, -1, -1):
                layer_error = prev_delta.dot(self.weights[layer+1].synaptic_weights.T)
                layer_delta = layer_error * self.activate(self.layer_outputs[layer], deriv=True)

                # Calculate how much to adjust the weights by
                if layer == 0:
                    layer_adjustment = training_set_inputs.T.dot(layer_delta)
                else:
                    layer_adjustment = self.layer_outputs[layer-1].T.dot(layer_delta)
                    
                layer_adjustments.insert(0, layer_adjustment)
                
                prev_delta = layer_delta

            # Adjust the weights.
            for layer in range(len(self.weights)):
                print(self.weights[layer].synaptic_weights)
                print(layer_adjustments[layer])
                
                self.weights[layer].synaptic_weights += layer_adjustments[layer]
            
            # print MSE
            if e % (epochs / 10) == 0:
                loss = np.mean((output - targets) ** 2)

                if 'last_loss' in locals() and last_loss < loss:
                    print("Train loss: ", loss, "  WARNING - Loss Increasing")
                else:
                    print("Train loss: ", loss)
                last_loss = loss
            
    
    
    def predict(self, X):
        return self.forward_prop(X)
    

In [32]:
input_data = np.array([[0,0,1],
                       [0,1,1],
                       [1,0,1],
                       [1,1,1]])
                
output_labels = np.array([[0],
                          [1],
                          [1],
                          [0]])

output_labels.shape

(4, 1)

In [33]:
layer1 = DenseLayer(4, 3)
layer2 = DenseLayer(1, 4)

nn = NN((layer1, layer2), 0.005)

[(3, 4), (4, 1)]


In [34]:
nn.weights

(<__main__.DenseLayer at 0x7f1d040787b8>,
 <__main__.DenseLayer at 0x7f1d04078d30>)

In [35]:
nn.train(input_data, output_labels, 50)

ValueError: operands could not be broadcast together with shapes (4,4) (4,3) 